In [1]:
#IMPORTS

#File IO
import os
import glob

#Data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Scikit learn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#Misc
from tqdm import tqdm


In [2]:
#HYPERPARMETERS

test_proportion = .2
max_images = 200

In [3]:
#GET TARGET DATA

def get_phase(file):
    data = np.load(file)
    bands = data['rad'].reshape(128 * 128, 16)
    cloud_phase = data['l2_cloud_phase'].flatten().reshape(-1, 1)
    return bands, cloud_phase

In [4]:
#CREATE DATASET

scaler = StandardScaler()
file_list = glob.glob('../ABI_Data_Scaled/*.npz')

X = []
y = []

for i, file in enumerate(file_list):
    features, target = get_phase(file)
    X.append(features)
    y.append(target)
    if i == max_images: break

X = scaler.fit_transform(np.concatenate(X, axis=0))
y = np.concatenate(y, axis=0).flatten()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_proportion, random_state=1)

In [5]:
# Train decision tree
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val, y_pred)

print(f"Validation Accuracy: {acc:.4f}")

Validation Accuracy: 0.8902


In [6]:

def iou_per_class(y_true, y_pred, num_classes=None):
    if num_classes is None:
        num_classes = max(np.max(y_true), np.max(y_pred)) + 1

    cm = confusion_matrix(y_true, y_pred, labels=np.arange(num_classes))
    intersection = np.diag(cm)
    ground_truth_set = cm.sum(axis=1)
    predicted_set = cm.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection

    iou = intersection / np.maximum(union, 1e-6)
    return iou

# Usage
ious = iou_per_class(y_val, y_pred)
for i, iou in enumerate(ious):
    print(f"Class {i}: IoU = {iou:.4f}")

Class 0: IoU = 0.7420
Class 1: IoU = 0.8335
Class 2: IoU = 0.7839
Class 3: IoU = 0.4357
Class 4: IoU = 0.8507


In [7]:
mean_iou = np.mean(ious)
print(f"Mean IoU: {mean_iou:.4f}")

Mean IoU: 0.7292
